# Library

In [1]:
import os,glob
import tensorflow as tf
import numpy as np

from scipy import signal, fftpack
from scipy.fft import fft, fftfreq
from scipy.signal import butter, filtfilt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import utils
utils.to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout, AveragePooling3D
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
load = "E:/Sekolah/Telkom/TA/LTSM/"
save = "E:/Sekolah/Telkom/TA/LTSM/Img/"
files_csv =[]
for root, dirs, files in os.walk(load):
    for file_name in files:
        if file_name.endswith(".csv"):
            file_load = os.path.join(root, file_name)
            files_csv.append(file_load)

In [3]:
x = []
y = []
for file_csv in files_csv:
    print(f"Head dari file {file_csv}:")
    df = pd.read_csv(file_csv)
    x.append(df['X'])
    y.append(df['Y'])
    print(df.head())
    print("\n")

Head dari file E:/Sekolah/Telkom/TA/LTSM/30%\A301.csv:
          X         Y
0  5.000000 -0.035156
1  5.015625 -0.035156
2  5.031250 -0.039062
3  5.046875 -0.007812
4  5.062500  0.117188


Head dari file E:/Sekolah/Telkom/TA/LTSM/30%\A302.csv:
          X         Y
0  5.000000 -0.050781
1  5.015625 -0.054688
2  5.031250 -0.046875
3  5.046875 -0.039062
4  5.062500  0.117188


Head dari file E:/Sekolah/Telkom/TA/LTSM/30%\A303.csv:
          X         Y
0  5.000000 -0.046875
1  5.015625 -0.042969
2  5.031250 -0.039062
3  5.046875 -0.007812
4  5.062500  0.117188


Head dari file E:/Sekolah/Telkom/TA/LTSM/30%\A304.csv:
          X         Y
0  5.000000 -0.058594
1  5.015625 -0.058594
2  5.031250 -0.062500
3  5.046875 -0.039062
4  5.062500  0.117188


Head dari file E:/Sekolah/Telkom/TA/LTSM/30%\A305.csv:
          X         Y
0  5.000000 -0.050781
1  5.015625 -0.054688
2  5.031250 -0.054688
3  5.046875 -0.046875
4  5.062500  0.117188


Head dari file E:/Sekolah/Telkom/TA/LTSM/30%\K301.csv:


# Parameter Filter, Apply Bandpass, Rectify Signal

In [4]:
lowcut = 2.0
highcut = 10.0
fs = 100.0
order = 5

# Apply a bandpass filter to the signal

In [5]:
i = 0

for file_csv in files_csv:
   
    df = pd.read_csv(file_csv)
    var = df['X'].values
    data = df['Y'].values
     
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='band')
    
    #Filter bandpass
    filtered_signal = signal.filtfilt(b, a, data)
    
    # Rectify signal
    rectified_signal = np.abs(filtered_signal)
    
    # One-sided FFT
    signal_fft = np.fft.fft(filtered_signal)
    signal_freq = np.fft.fftfreq(len(filtered_signal), 1/fs)
    n = len(filtered_signal)
    freq = signal_freq[0:int(n/2)]
    fft_amp = np.abs(signal_fft)[0:int(n/2)]/n*2
    
    # One-sided PSD
    psd = (1/(fs*n)) * np.square(np.abs(signal_fft[:n//2]))
    freq_psd = signal_freq[0:int(n/2)]
    psd_one_side = psd[0:int(n/2)]
    
    # processed_csv_file = file_csv.replace(".csv", "_processed.csv")
    # df.to_csv(processed_csv_file, index=False)
    
    # fig, axs = plt.subplots(2, 2, figsize=(10,10))

    # axs[0, 0].plot(var, filtered_signal)
    # axs[0, 0].set_title(file_csv)
    # axs[0, 0].set_xlabel('Time (us)')
    # axs[0, 0].set_ylabel('Amplitude (V)')

    # axs[0, 1].plot(var, rectified_signal)
    # axs[0, 1].set_title(file_csv)
    # axs[0, 1].set_xlabel('Time (us)')
    # axs[0, 1].set_ylabel('Amplitude (V)')

    # axs[1, 0].plot(freq, fft_amp)
    # axs[1, 0].set_title(file_csv)
    # axs[1, 0].set_xlabel('Frequency (MHz)')
    # axs[1, 0].set_ylabel('Amplitude Power/Frequency (dB/Hz)')

    # axs[1, 1].plot(freq_psd, psd_one_side)
    # axs[1, 1].set_title(file_csv)
    # axs[1, 1].set_xlabel('Frequency (MHz)')
    # axs[1, 1].set_ylabel('Power/Frequency (dB/Hz)')

    # save_img = file_csv+".png"
    # plt.savefig(save_img)
    
    
    

In [6]:
# os.chdir(load)
# if not os.path.exists(load+"Process"):
#         os.makedirs(load+"Process")
#         print("'data' folder created")
        
# for root, dirs, files in os.walk(load):
#         for filename in files:
#             if filename.endswith("_processed.csv"):
#                 original_file = os.path.join(root, filename)                
#                 if not "Process/" in original_file:
#                     print("Moved '",original_file," to ","./Process/"+filename)
#                     os.rename(original_file, "./Process/"+filename)

In [7]:
load2 = "E:/Sekolah/Telkom/TA/LTSM/Process/"
data = []
labels = []

for subfolder_name in os.listdir(load2):
    subfolder_path = os.path.join(load2, subfolder_name)
    if os.path.isdir(subfolder_path):
        # List untuk menyimpan data dan label dari subfolder
        subfolder_data = []
        subfolder_labels = []
        
        for file_name in os.listdir(subfolder_path):
            if file_name.endswith(".csv"):
                file_path = os.path.join(subfolder_path, file_name)
                df = pd.read_csv(file_path)
                features = df.iloc[:, :-1].values
                target = df.iloc[:, -1].values
                subfolder_data.append(features)
                subfolder_labels.append(target)
                
        subfolder_data = np.concatenate(subfolder_data)
        subfolder_labels = np.concatenate(subfolder_labels)
        data.append(subfolder_data)
        labels.append(subfolder_labels)

In [87]:
print(features)

[[  5.      ]
 [  5.015625]
 [  5.03125 ]
 ...
 [199.953125]
 [199.96875 ]
 [199.984375]]


In [8]:
data = np.concatenate(data)
labels = np.concatenate(labels)
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [10]:
# Membangun model LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64))
model.add(Dropout(0.3))
model.add(Dense(len(label_encoder.classes_), activation='tanh'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))
model.summary

<bound method Model.summary of <keras.src.engine.sequential.Sequential object at 0x000001BA973A4190>>

In [11]:
# Mengompilasi model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
# # Melatih model
# history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

In [13]:
# _, accuracy = model.evaluate(X_test, y_test)
# print(f"Akurasi: {accuracy * 100:.2f}%")

In [14]:
# model.save("model.h5")

In [15]:
model = load_model('model.h5')

In [52]:
df=pd.read_csv('E:/Sekolah/Telkom/TA/LTSM/Process/30%/A301_processed.csv')
df.head()

,X,Y
0,5.000000,-0.035156
1,5.015625,-0.035156
2,5.031250,-0.039062
3,5.046875,-0.007812
4,5.062500,0.117188


In [53]:
X=df.values

In [48]:
X=X[:]

In [54]:
print(X)

[[ 5.00000000e+00 -3.51562500e-02]
 [ 5.01562500e+00 -3.51562500e-02]
 [ 5.03125000e+00 -3.90625000e-02]
 ...
 [ 1.99953125e+02 -3.90625000e-03]
 [ 1.99968750e+02  1.97656250e+00]
 [ 1.99984375e+02  1.51953125e+00]]


In [58]:
y=np.array(df['Y'])
Y=utils.to_categorical(y)
Y.shape
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [60]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [61]:
X_train.shape
X_test.shape

(2496, 2, 1)

In [62]:
ypred=model.predict((X_test[:,::4]-X_test.mean())/X_test.std())
ypred.shape

78/78 [==============================] - 1s 4ms/step


(2496, 634)

In [63]:
yp=np.zeros((Y_test.shape[0]))
yo=np.ones((Y_test.shape[0]))

In [64]:
for i in range(Y_test.shape[0]):
    yp[i]=np.argmax(ypred[i])+1
    yo[i]=np.argmax(Y_test[i])

In [65]:
yp.shape

(2496,)

In [66]:
yo.shape

(2496,)

In [67]:
np.unique(yo)

array([0., 1.])

In [68]:
np.unique(yp)

array([246., 343., 367.])

In [69]:
for i in range(Y_test.shape[0]):
    if yo[i]!=1:
        yo[i]=0
    if yp[i]!=1:
        yp[i]=0

In [70]:
np.unique(yo)

array([0., 1.])

In [72]:
np.unique(yp)

array([0.])

In [73]:
from sklearn.metrics import accuracy_score

In [74]:
accuracy_score(yo,yp)

0.9995993589743589

In [75]:
ypred1=model.predict((X_train[:,::4]-X_train.mean())/X_train.std())
ypred1.shape

312/312 [==============================] - 1s 3ms/step


(9984, 634)

In [76]:
yp1=np.zeros((Y_train.shape[0]))
yo1=np.ones((Y_train.shape[0]))

In [77]:
for i in range(Y_train.shape[0]):
    yp1[i]=np.argmax(ypred1[i])+1
    yo1[i]=np.argmax(Y_train[i])

In [78]:
yp1.shape

(9984,)

In [79]:
yo1.shape

(9984,)

In [80]:
np.unique(yo1)

array([0., 1.])

In [81]:
np.unique(yp1)

array([246., 343., 367.])

In [82]:
#conversion of classes
for i in range(Y_train.shape[0]):
    if yo1[i]!=1:
        yo1[i]=0
    if yp1[i]!=1:
        yp1[i]=0

In [83]:
np.unique(yo1)

array([0., 1.])

In [84]:
np.unique(yp1)

array([0.])

In [85]:
accuracy_score(yo1,yp1)

0.9998998397435898